In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

<h1>Составление талбицы с CPI</h1>

In [2]:
url = "https://www.usinflationcalculator.com/inflation/consumer-price-index-and-annual-percent-changes-from-1913-to-2008/"

In [3]:
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, "html.parser")

In [4]:
table = soup.find('table')
rows = table.tbody.findAll('tr')

In [5]:
years = list()
cpis = list()

In [6]:
for row in rows:
    year = row.findAll('td')[0].get_text()
    if year.isdigit() and int(year) <= 2017:
        years.append(int(year))
        cpis.append(float(row.findAll('td')[13].get_text()))

In [7]:
cpi_table = pd.DataFrame({
    "year": years,
    "avg_annual_cpi": cpis
})

<h1>Запись таблицы c CPI в файл</h1>

In [8]:
cpi_table.to_csv("../data/Lab1_Koren_CPI.csv", sep=',')

In [9]:
def get_real_value(nominal_amt, old_cpi, new_cpi):
    real_value = (nominal_amt * new_cpi) / old_cpi
    return real_value

In [10]:
CPI_2017 = float(cpi_table[cpi_table['year'] == 2017]['avg_annual_cpi'])

C:\Users\qwerty\AppData\Local\Temp\ipykernel_12840\793858478.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  CPI_2017 = float(cpi_table[cpi_table['year'] == 2017]['avg_annual_cpi'])


<h1></h1>

<h1>Считываем данных из csv файла, где хранится информация о фильмах</h1>

In [11]:
movies_table = pd.read_csv('../data/Dataset1.csv')

<h1></h1>

<h1>Прописываем в пустые ячейки "unknown"</h1>

In [12]:
movies_table.isna().sum()

color                          18
Director_Name                 171
num_Critic_for_reviews        116
duration                       84
director_Facebook_likes       171
actor_3_Facebook_likes         90
actor_2_name                   80
Actor_1_Facebook_likes         75
gross                         939
genres                         69
actor_1_name                   75
movie_Title                    69
num_voted_users                69
 cast_total_facebook_likes     69
actor_3_name                   90
facenumber_in_poster           82
plot_keywords                 216
movie_imdb_link                69
num_user_for_reviews           87
language                       82
country                        72
content_rating                366
budget                        549
title_year                    174
actor_2_facebook_likes         80
imdb_score                     69
aspect_ratio                  389
movie_facebook_likes;          69
dtype: int64

In [13]:
columns_to_replace =['title_year', 'gross', 'budget']

for row in columns_to_replace:
    movies_table[row] = movies_table[row].fillna('unknown')

In [14]:
movies_table.isna().sum()

color                          18
Director_Name                 171
num_Critic_for_reviews        116
duration                       84
director_Facebook_likes       171
actor_3_Facebook_likes         90
actor_2_name                   80
Actor_1_Facebook_likes         75
gross                           0
genres                         69
actor_1_name                   75
movie_Title                    69
num_voted_users                69
 cast_total_facebook_likes     69
actor_3_name                   90
facenumber_in_poster           82
plot_keywords                 216
movie_imdb_link                69
num_user_for_reviews           87
language                       82
country                        72
content_rating                366
budget                          0
title_year                      0
actor_2_facebook_likes         80
imdb_score                     69
aspect_ratio                  389
movie_facebook_likes;          69
dtype: int64

<h1></h1>

<h1>Вычисление валовой и бюджетной стоимости в реальной сумме</h1>

In [15]:
real_domestic_gross = []
real_budget_values = []

In [16]:
for index, row in movies_table.iterrows():
    gross = row['gross']
    budget = row['budget']
    year = row['title_year']
    if year == "unknown"  or gross == "unknown" or budget == "unknown":
        real_domestic_gross.append(gross)
        real_budget_values.append(budget)
        continue
    cpi = float(cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'])
    
    real_gross = get_real_value(gross, cpi, CPI_2017)
    real_budget = get_real_value(budget, cpi, CPI_2017)
    real_domestic_gross.append(real_gross)
    real_budget_values.append(real_budget)
movies_table["real_domestic_gross"] = real_domestic_gross
movies_table["real_budget"] = real_budget_values

C:\Users\qwerty\AppData\Local\Temp\ipykernel_12840\1815066857.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cpi = float(cpi_table[cpi_table['year'] == int(year)]['avg_annual_cpi'])


<h1></h1>

<h1>Вычисление ROI и абсолютной прибыли</h1>

In [17]:
profits = []
roi_vals = []

In [18]:
for index, row in movies_table.iterrows():
    if row["real_domestic_gross"] == "unknown" or row['real_budget'] == "unknown":
        profits.append("unknown")
        roi_vals.append("unknown")
        continue
    profit = row["real_domestic_gross"] - row['real_budget']
    budget = row['real_budget']
    num = profit - budget
    den = budget
    roi = (num / den) * 100
    
    profits.append(profit)
    roi_vals.append(roi)

movies_table['profit'] = profits
movies_table['roi'] = roi_vals

<h1></h1>

<h1>Удаление дубликатов</h1>

In [19]:
movies_table.duplicated().sum()

45

In [20]:
movies_table = movies_table.drop_duplicates().reset_index(drop=True)

In [21]:
movies_table.duplicated().sum()

0

<h1></h1>

<h1>Гипотезы</h1>

1. Большая часть фильмов окупилась 
2. Action - самый чаcтый жанр
3. Средняя оценка фильмов 6.5 +- 0.2


<h2>Проверка первой гипотезы</h2>

In [23]:
count_profit_films = 0
count_films = 0
for index, row in movies_table.iterrows():
    if row['profit'] == "unknown":
        continue
    if row['profit'] > 0:
        count_profit_films += 1
    count_films += 1

print((count_profit_films * 100) / count_films)

52.584623458409865


Из волученного результата видно, что 52.58% фильмов окупились.

<h2>Проверка второй гипотезы</h2>

In [24]:
popularity_of_genres = dict()
for index, row in movies_table.iterrows():
    if not isinstance(row['genres'], str):
        continue
    for genre in row['genres'].split('|'):
        if genre not in popularity_of_genres:
            popularity_of_genres[genre] = 1
        else:
            popularity_of_genres[genre] += 1

popularity_of_genres = pd.DataFrame.from_dict(popularity_of_genres, orient='index', columns=['popularity'])
        

In [25]:
popularity_of_genres = popularity_of_genres.sort_values(by=['popularity'], ascending=False)
popularity_of_genres.head()

,popularity
Drama,2543
Comedy,1826
Thriller,1389
Action,1134
Romance,1081


По таблице видно, что гипотеза оказалась ошибочной, и самый частый жанр - Drama

<h2>Проверка третий гипотезы</h2>

In [26]:
print(movies_table['imdb_score'].mean(numeric_only=True))

6.438498985801217


Средняя оценка за фильм 6.4. Гипотеза подтверждена.